In [1]:
import os

os.chdir("..")
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import tqdm.auto as tqdm

from modular_splicing.dataset.basic_dataset import basic_dataset

In [4]:
barcode_length = 4

In [5]:
marginal_dist = np.zeros((4 ** barcode_length, 4), np.int)
for path in ["dataset_test_0.h5", "dataset_train_all.h5"]:
    dset = basic_dataset(
        path=path,
        cl=barcode_length * 2,
        cl_max=10_000,
        sl=5000,
    )
    for x, _ in tqdm.tqdm(dset, desc=path):
        x = x.astype(np.int)
        barcode_bases, starting_points = np.arange(barcode_length), np.arange(len(x) - 2 * barcode_length)
        barcode_bases, starting_points = np.meshgrid(barcode_bases, starting_points)
        x_barcode = x[barcode_bases + starting_points]
        x_mask = x_barcode.sum(-1).all(-1)
        x_barcode = x_barcode.argmax(-1)
        x_barcode_codes = x_barcode @ (4 ** np.arange(barcode_length))
        np.add.at(
            marginal_dist,
            (x_barcode_codes[x_mask], x[barcode_length:x.shape[0]-barcode_length][x_mask].argmax(-1)),
            1
        )

In [6]:
barcode_frequencies = marginal_dist.sum(1)
relative_freqs = marginal_dist / barcode_frequencies[:,None]

In [7]:
relative_barcode_freqs = barcode_frequencies / barcode_frequencies.sum()

In [8]:
entropy = (relative_barcode_freqs * ((-relative_freqs * np.log(relative_freqs)) / np.log(2)).sum(1)).sum()

In [9]:
entropy

1.9115757921758971